In [ ]:
from dotenv import load_dotenv                                                  # type: ignore
load_dotenv()

import ollama                                                                   # type: ignore
response = ollama.web_search("What is Ollama?")
print(response)

In [ ]:
import os
from dotenv import load_dotenv                                                  # type: ignore
load_dotenv()

from ollama import chat, web_fetch, web_search                                  # type: ignore


available_tools = {'web_search': web_search, 'web_fetch': web_fetch}

messages = [{'role': 'user', 'content': "what is ollama's new engine"}]

while True:

    MODEL = 'qwen3:0.6b' #'qwen3:4b'

    response = chat(model=MODEL, messages=messages, tools=[web_search, web_fetch], think=True)

    if response.message.thinking: print('Thinking: ', response.message.thinking)

    if response.message.content: print('Content: ', response.message.content)
    
    messages.append(response.message)
    
    if not response.message.tool_calls: break

    print('Tool calls: ', response.message.tool_calls)
    
    for tool_call in response.message.tool_calls: function_to_call = available_tools.get(tool_call.function.name)

    if function_to_call:
        args = tool_call.function.arguments
        result = function_to_call(**args)
        print('Result: ', str(result)[:200]+'...')
        # Result is truncated for limited context lengths
        messages.append({'role': 'tool', 'content': str(result)[:2000 * 4], 'tool_name': tool_call.function.name})
    else:
        messages.append({'role': 'tool', 'content': f'Tool {tool_call.function.name} not found', 'tool_name': tool_call.function.name})